# Functions

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split

import os

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape,Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split, LeaveOneOut

import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Code

In [2]:
def build_autoencoder(input_shape=(256, 256, 1)):
    input_img = Input(shape=input_shape, name='input_image')

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    encoded = Flatten(name='encoded_features')(x)

    radius_pred = Dense(1, activation='linear', name='radius_output')(encoded)
    ccol_pred = Dense(1, activation='linear', name='ccol_output')(encoded)
    crow_pred = Dense(1, activation='linear', name='crow_output')(encoded)

    x = Dense(1024, activation='relu')(encoded)
    x = Reshape((4, 4, 64))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)             
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)              
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)            
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)             
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)              
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='image_output')(x)

    print("radius_output shape:", radius_pred.shape)
    print("image shape:", decoded.shape)
    print("ccol shape:", ccol_pred.shape)
    print("crow shape:", crow_pred.shape)

    autoencoder = Model(inputs=input_img, outputs=[decoded, radius_pred, ccol_pred, crow_pred], name='autoencoder')
    return autoencoder



def train_autoencoder(autoencoder, x_train, radii_train, x_val, radii_val, ccol_train, crow_train, ccol_val, crow_val, epochs=50, batch_size=32):
    autoencoder.compile(
        optimizer='adam',
        loss={
            'image_output': 'mse',       
            'radius_output': 'mse',       
            'ccol_output': 'mse',        
            'crow_output': 'mse'          
        }
    )

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = autoencoder.fit(
    x_train,
    {
        'image_output': x_train,
        'radius_output': radii_train,
        'ccol_output': ccol_train,
        'crow_output': crow_train
    },
    validation_data=(
        x_val,
        {
            'image_output': x_val,
            'radius_output': radii_val,
            'ccol_output': ccol_val,
            'crow_output': crow_val
        }
    ),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping]
)


    autoencoder.summary()

    return history



In [3]:



df = pd.read_excel('../combined_results.xlsx')
df['Image Path'] = df['Image Path'].apply(lambda x: '.' + x)
image_paths = df['Image Path'].tolist()  
radii = df['circle_radius'].values   
ccols = df['center_y(ccol)'].values 
crows = df['center_x(crow)'].values


radii = radii.reshape(-1, 1)  # Ensure shape: (num_samples, 1)
ccols = ccols.reshape(-1, 1) 
crows = crows.reshape(-1, 1)  


img_height, img_width = 256, 256  
images = []

for path in image_paths:
  
    img = load_img(path, target_size=(img_height, img_width), color_mode='grayscale')
    img_array = img_to_array(img) / 255.0  
    images.append(img_array)


images = np.array(images)  # Shape: (num_samples, img_height, img_width, 1)


x_train, x_test, radii_train, radii_test,ccol_train,ccol_test,crow_train,crow_test = train_test_split(
    images, radii,ccols,crows, test_size=0.2, random_state=42
)



autoencoder = build_autoencoder(input_shape=(img_height, img_width, 1))  
autoencoder.compile(
        optimizer='adam',
        loss={
            'image_output': 'mse',
            'radius_output': 'mse',
            'ccol_output': 'mse',
            'crow_output': 'mse'
        }
    )

print("x_train shape:", x_train.shape)
print("radii_train shape:", radii_train.shape)
print("x_val shape:", x_test.shape)
print("radii_val shape:", radii_test.shape)
print("ccol_train shape:", ccol_train.shape)
print("ccol_val shape:", ccol_test.shape) 
print("crow_train shape:", crow_train.shape)
print("crow_val shape:", crow_test.shape) 

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = autoencoder.fit(
    x_train,
    {
        'image_output': x_train, 
        'radius_output': radii_train,
        'ccol_output':ccol_train,
        'crow_output':crow_train
    },
    validation_data=(x_test,{
        'image_output': x_test,
        'radius_output': radii_test,
        'ccol_output': ccol_test,
        'crow_output': crow_test
    }),
    epochs=50,
    batch_size=32,
    verbose=1
)


# autoencoder.save('diffraction_autoencoder_center.h5')


test_images = x_test[:10]  
reconstructed_images, predicted_radii,pred_ccol,pred_crow = autoencoder.predict(test_images)


print("Predicted Radii:", predicted_radii.flatten())


radius_output shape: (None, 1)
image shape: (None, 256, 256, 1)
ccol shape: (None, 1)
crow shape: (None, 1)
x_train shape: (177, 256, 256, 1)
radii_train shape: (177, 1)
x_val shape: (45, 256, 256, 1)
radii_val shape: (45, 1)
ccol_train shape: (177, 1)
ccol_val shape: (45, 1)
crow_train shape: (177, 1)
crow_val shape: (45, 1)
Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mse_2/sub/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\Tautvydas\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "c:\Users\Tautvydas\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1987, in _